In [9]:
import praw
import json
import pandas as pd
import time

In [5]:
app_id = 'REDACTED'
secret = 'REDACTED'
agent = 'REDACTED'

reddit = praw.Reddit(client_id=app_id,
                     client_secret=secret,
                     user_agent=agent)

reddit.read_only = True

In [ ]:
crawl_plan = {
    "nike":100
}

In [16]:
s = reddit.subreddit('nike')
s.controversial()

In [18]:
def scrape_subreddit(subreddit='nike', n_rows=25, sort_by='hot', time='all'):
    """
    gets n_rows data from a subreddit.
    subreddit - string, the name of the subreddit (default: "nike")
    n_rows - int, the number of rows in desired output dataset (default: 25)
    sort_by - one of "hot", "top", "new", or "controversial" (default: "hot")
    time - "all", "day", "hour", "month", "week", "year" (default: "all")
    """
    #n_rows = 100
    cols = ['title', 'submission_id', 'url', 'score', 'upvote_ratio', 'is_text_post', 'content', 'comments']
    output = pd.DataFrame(columns=cols)
    if sort_by == 'hot':
        sub_iterator = reddit.subreddit(subreddit).hot(time)
    elif sort_by == 'top':
        sub_iterator = reddit.subreddit(subreddit).top(time)
    elif sort_by == 'controversial':
        sub_iterator = reddit.subreddit(subreddit).controversial(time)
    elif sort_by == 'new':
        sub_iterator = reddit.subreddit(subreddit).new(time)
    
    for i, submission in enumerate(sub_iterator):
        print('getting ', i, 'th row...')
        row = pd.DataFrame(columns=cols, index=[i])
        row['title'] = submission.title
        row['submission_id'] = submission.id
        row['url'] = submission.url
        row['score'] = submission.score
        row['upvote_ratio'] = submission.upvote_ratio
        if submission.is_self:
            is_text = True
            content = submission.selftext
        else:
            is_text = False
            content = submission.url
        row['is_text_post'] = is_text
        row['content'] = content
        submission.comments.replace_more(limit=0)
        comment_chain = []
        for comment in submission.comments.list():
            comment_chain.append(comment.body)
        row['comments'] = json.dumps(comment_chain)
        output = pd.concat([output, row])
        time.sleep(1)
        if len(output) >= n_rows:
            print('got ', len(output), ' rows. Finishing!')
            break

    return output

output = scrape_subreddit()
output

getting  0 th row...
getting  1 th row...
getting  2 th row...
getting  3 th row...
getting  4 th row...
getting  5 th row...
getting  6 th row...
getting  7 th row...
getting  8 th row...
getting  9 th row...
getting  10 th row...
getting  11 th row...
getting  12 th row...
getting  13 th row...
getting  14 th row...
getting  15 th row...
getting  16 th row...
getting  17 th row...
getting  18 th row...
getting  19 th row...
getting  20 th row...
getting  21 th row...
getting  22 th row...
getting  23 th row...
getting  24 th row...
getting  25 th row...
got  26  rows. Finishing!


,title,submission_id,url,score,upvote_ratio,is_text_post,content,comments
0,Tips for identifying Nike Product,47fex4,https://www.reddit.com/r/Nike/comments/47fex4/...,21,1.00,True,Please note that any product from a previous s...,"[""If you are posting a photo of a product you ..."
1,Just got these!,nkmf3l,https://i.redd.it/yp5hfa6pt8171.jpg,107,0.99,False,https://i.redd.it/yp5hfa6pt8171.jpg,"[""What are they called?"", ""Nice I bet they r c..."
2,What do y’all think about these? Picked up for...,nkwek0,https://i.redd.it/9atsx28p6b171.jpg,2,1.00,False,https://i.redd.it/9atsx28p6b171.jpg,[]
3,Yeezy Boost 350 baratos cómodos y de buena cal...,nkupj2,https://youtube.com/watch?v=z5ZFLjIa8rw&featur...,2,0.75,False,https://youtube.com/watch?v=z5ZFLjIa8rw&featur...,"[""whats app +8618060442591""]"
4,Any one knows the name of his sweatshirt ?,nktz3m,https://i.redd.it/dh3ff40vna171.jpg,2,1.00,False,https://i.redd.it/dh3ff40vna171.jpg,"[""You\u2019d have to ask his little brother."",..."
5,Can anyone ID these? They are 20+ years old,nkteln,https://www.reddit.com/gallery/nkteln,2,1.00,False,https://www.reddit.com/gallery/nkteln,"[""Retro 18s"", ""I can smell these"", ""They are 1..."
6,Has anyone else noticed the jumpman logo doing...,nkkp0w,https://i.redd.it/t3smigyl88171.jpg,9,0.76,False,https://i.redd.it/t3smigyl88171.jpg,"[""it\u2019s the angle."", ""I checked the Snkrs ..."
7,This pair is quite photogenic!,njvtz5,https://i.imgur.com/ZrBy3HM.jpg,277,0.99,False,https://i.imgur.com/ZrBy3HM.jpg,"[""Colourway?"", ""ughh so fireee"", ""I got get me..."
8,Son notices what grown-ups don't.,nkvzuv,http://rover.ebay.com/rover/1/711-53200-19255-...,1,1.00,False,http://rover.ebay.com/rover/1/711-53200-19255-...,[]
9,Anyone know where I can get this hoodie? I got...,nkvfbf,https://i.redd.it/y05m7qp0za171.jpg,1,1.00,False,https://i.redd.it/y05m7qp0za171.jpg,"[""Outlets""]"
